<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/mrkim21apps/Oxford3Kaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio gtts

In [ ]:
!pip install pydub
!apt-get install ffmpeg

In [ ]:
import gradio as gr
from gtts import gTTS
import pandas as pd
from pydub import AudioSegment
import io
import numpy as np

# Read the CSV data directly for demonstration purposes
# Here you should load your actual data

csv_file = "/content/Oxford3K.csv"

# Read the CSV file
data = pd.read_csv(csv_file)

df = pd.DataFrame(data)


def generate_speech(x, y):
    # Ensure x and y are integers
    x, y = int(x), int(y)

    # Create an empty audio segment for padding
    combined_audio = AudioSegment.silent(duration=1000)  # Starting with 1 second of silence for padding

    # Generate speech for each entry in the specified range
    for index, row in df.iloc[x-1:y].iterrows():
        sentence = f"Number {row['SID']}. {row['WORD']} is a {row['POS']}."
        tts = gTTS(text=sentence, lang='en')
        mp3_fp = io.BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        sentence_audio = AudioSegment.from_file(mp3_fp, format='mp3')
        # Add the sentence audio and 2 seconds of silence
        combined_audio += sentence_audio + AudioSegment.silent(duration=2000)

    # Export the combined audio to a BytesIO object and return the bytes
    mp3_io = io.BytesIO()
    combined_audio.export(mp3_io, format='mp3')
    mp3_io.seek(0)
    return mp3_io.read()

# Define the Gradio interface
iface = gr.Interface(
    fn=generate_speech,
    inputs=[
        gr.Number(label="Start Sentence Number (x)"),  # Assume the `default` parameter is removed if it causes an error
        gr.Number(label="End Sentence Number (y)")
    ],
    outputs=gr.Audio(label="Generated Speech"),
    title="Speech Generator",
    description="Generate speech from the CSV data. Specify the start and end sentence numbers."
)


# Launch the Gradio app with sharing enabled

iface.launch(share=True, debug=True)

# By category (option)

In [ ]:
import gradio as gr
from gtts import gTTS
import pandas as pd
from pydub import AudioSegment
import io

# Load your CSV data into 'df'
# df = pd.read_csv('path_to_your_csv.csv')

def generate_speech_for_level(level):
    # Find the range of SIDs for the selected level
    level_df = df[df['LEVEL'] == level]
    if not level_df.empty:
        x = level_df['SID'].min()
        y = level_df['SID'].max()
        return generate_speech(x, y)
    else:
        return "No data found for the selected level.", 22050

def generate_speech(x, y):
    # Ensure x and y are integers
    x = int(x)
    y = int(y)

    # Create an empty audio segment for padding
    combined_audio = AudioSegment.silent(duration=1000)  # Starting with 1 second of silence for padding

    # Generate speech for each entry in the specified range
    for index, row in df.iloc[x-1:y].iterrows():
        sentence = f"Number {row['SID']}. {row['WORD']} is {row['POS']}."
        tts = gTTS(text=sentence, lang='en')
        mp3_fp = io.BytesIO()
        tts.write_to_fp(mp3_fp)
        mp3_fp.seek(0)
        sentence_audio = AudioSegment.from_file(mp3_fp, format='mp3')
        combined_audio += sentence_audio + AudioSegment.silent(duration=2000)

    mp3_io = io.BytesIO()
    combined_audio.export(mp3_io, format='mp3')
    mp3_io.seek(0)
    return mp3_io.read()

with gr.Blocks() as demo:
    with gr.Row():
        b2_button = gr.Button("B2 Level")
        c1_button = gr.Button("C1 Level")
    with gr.Row():
        x_input = gr.Number(label="Start Sentence Number (x)")
        y_input = gr.Number(label="End Sentence Number (y)")
        generate_button = gr.Button("Generate Speech")

    audio_output = gr.Audio(label="Generated Speech")

    b2_button.click(fn=lambda: generate_speech_for_level("B2"), inputs=[], outputs=audio_output)
    c1_button.click(fn=lambda: generate_speech_for_level("C1"), inputs=[], outputs=audio_output)
    generate_button.click(fn=generate_speech, inputs=[x_input, y_input], outputs=audio_output)

demo.launch(share=True)


In [ ]:
import gradio as gr
from gtts import gTTS
import pandas as pd
from pydub import AudioSegment
import io

# Assuming 'df' is your DataFrame loaded with the CSV data

def update_sid_range_from_level(b2_checked, c1_checked):
    levels = []
    if b2_checked:
        levels.append("B2")
    if c1_checked:
        levels.append("C1")

    # Find the range of SIDs for the selected levels
    if levels:
        level_df = df[df['LEVEL'].isin(levels)]
        x = level_df['SID'].min() if not level_df.empty else "Not Available"
        y = level_df['SID'].max() if not level_df.empty else "Not Available"
    else:
        x, y = "Select a level", "Select a level"

    return str(x), str(y)

def generate_speech(x, y):
    # No need to check if x and y are digits since they're already integers
    x = int(x)  # This conversion is now redundant but kept for clarity
    y = int(y)  # Same as above

    combined_audio = AudioSegment.silent(duration=1000)  # Start with silence for padding

    if x <= y:
        for _, row in df.iloc[x-1:y].iterrows():
            sentence = f"Number {row['SID']}. {row['WORD']} is {row['POS']}."
            tts = gTTS(text=sentence, lang='en')
            mp3_fp = io.BytesIO()
            tts.write_to_fp(mp3_fp)
            mp3_fp.seek(0)
            sentence_audio = AudioSegment.from_file(mp3_fp, format="mp3")
            combined_audio += sentence_audio + AudioSegment.silent(duration=2000)

    mp3_io = io.BytesIO()
    combined_audio.export(mp3_io, format="mp3")
    mp3_io.seek(0)
    return mp3_io.getvalue()


with gr.Blocks() as demo:
    b2_checkbox = gr.Checkbox(label="B2 Level", value=False)
    c1_checkbox = gr.Checkbox(label="C1 Level", value=False)
    x_output = gr.Textbox(label="Start SID (x)", value="Select a level", interactive=False)
    y_output = gr.Textbox(label="End SID (y)", value="Select a level", interactive=False)
    x_input = gr.Number(label="Enter Start SID (x)", value=1)
    y_input = gr.Number(label="Enter End SID (y)", value=2)
    generate_button = gr.Button("Generate Speech")
    audio_output = gr.Audio(label="Generated Speech")

    def update_checkboxes(b2, c1):
        return update_sid_range_from_level(b2, c1)

    # Attach the change event to each checkbox, calling the same function
    b2_checkbox.change(fn=update_checkboxes, inputs=[b2_checkbox, c1_checkbox], outputs=[x_output, y_output])
    c1_checkbox.change(fn=update_checkboxes, inputs=[b2_checkbox, c1_checkbox], outputs=[x_output, y_output])

    generate_button.click(fn=generate_speech, inputs=[x_input, y_input], outputs=audio_output)


demo.launch(share=True, debug=True)
